# API de Clasificación de Cáncer de Mama - Despliegue en Google Colab

Este notebook demuestra cómo desplegar y utilizar nuestra API de clasificación de cáncer de mama en Google Colab. Seguiremos un proceso paso a paso para configurar el entorno, entrenar el modelo y realizar predicciones.

## 1. Configuración del Entorno

Primero, instalemos las dependencias necesarias:

In [ ]:
!pip install fastapi uvicorn scikit-learn numpy pyngrok

## 2. Código de la API

Ahora crearemos el archivo principal de la API:

In [ ]:
%%writefile main.py
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
import uvicorn
import pickle
import numpy as np
import logging
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

app = FastAPI(title="API de Clasificación de Cáncer de Mama",
              description="API para clasificación de cáncer de mama usando el conjunto de datos Wisconsin Breast Cancer",
              version="1.0.0")

class EntradaCaracteristicas(BaseModel):
    caracteristicas: list
    
    class Config:
        schema_extra = {
            "example": {
                "caracteristicas": [17.99, 10.38, 122.8, 1001.0, 0.1184, 0.2776, 0.3001, 0.1471, 0.2419, 0.07871, 1.095, 0.9053, 8.589, 153.4, 0.006399, 0.04904, 0.05373, 0.01587, 0.03003, 0.006193, 25.38, 17.33, 184.6, 2019.0, 0.1622, 0.6656, 0.7119, 0.2654, 0.4601, 0.1189]
            }
        }

class RespuestaPrediccion(BaseModel):
    prediccion: int
    etiqueta_prediccion: str
    probabilidad: float

RUTA_MODELO = "modelo_cancer_mama.pkl"

def entrenar_y_guardar_modelo():
    logger.info("Cargando conjunto de datos de cáncer de mama")
    datos = load_breast_cancer()
    X = datos.data
    y = datos.target
    nombres_caracteristicas = datos.feature_names
    nombres_objetivos = datos.target_names
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)
    
    y_pred = modelo.predict(X_test)
    precision = accuracy_score(y_test, y_pred)
    logger.info(f"Modelo entrenado con precisión: {precision:.4f}")
    
    datos_modelo = {
        "modelo": modelo,
        "nombres_caracteristicas": nombres_caracteristicas,
        "nombres_objetivos": nombres_objetivos
    }
    
    with open(RUTA_MODELO, 'wb') as f:
        pickle.dump(datos_modelo, f)
    
    return datos_modelo

def obtener_modelo():
    if not os.path.exists(RUTA_MODELO):
        return entrenar_y_guardar_modelo()
    
    try:
        with open(RUTA_MODELO, 'rb') as f:
            datos_modelo = pickle.load(f)
        return datos_modelo
    except Exception as e:
        return entrenar_y_guardar_modelo()

@app.get("/")
def leer_raiz():
    return {"mensaje": "Bienvenido a la API de Clasificación de Cáncer de Mama"}

@app.post("/predecir", response_model=RespuestaPrediccion)
def predecir(datos_entrada: EntradaCaracteristicas, datos_modelo=Depends(obtener_modelo)):
    try:
        modelo = datos_modelo["modelo"]
        nombres_caracteristicas = datos_modelo["nombres_caracteristicas"]
        nombres_objetivos = datos_modelo["nombres_objetivos"]
        
        if len(datos_entrada.caracteristicas) != len(nombres_caracteristicas):
            raise HTTPException(status_code=400, detail=f"Se esperaban {len(nombres_caracteristicas)} características")
        
        caracteristicas = np.array(datos_entrada.caracteristicas).reshape(1, -1)
        prediccion = modelo.predict(caracteristicas)[0]
        probabilidad = modelo.predict_proba(caracteristicas)[0][prediccion]
        
        return {
            "prediccion": int(prediccion),
            "etiqueta_prediccion": nombres_objetivos[prediccion],
            "probabilidad": float(probabilidad)
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/caracteristicas")
def obtener_caracteristicas(datos_modelo=Depends(obtener_modelo)):
    return {"nombres_caracteristicas": datos_modelo["nombres_caracteristicas"].tolist()}

## 3. Iniciar el Servidor

Ahora iniciaremos el servidor FastAPI y lo expondremos usando ngrok:

In [ ]:
from pyngrok import ngrok

# Iniciar el servidor FastAPI en segundo plano
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &

# Crear un túnel HTTP con ngrok
public_url = ngrok.connect(8000)
print(f'La API está disponible en: {public_url}')

## 4. Realizar Predicciones

Ahora podemos hacer predicciones usando la API:

In [ ]:
import requests
import json

# Cargar datos de ejemplo
from sklearn.datasets import load_breast_cancer
datos = load_breast_cancer()
ejemplo = datos.data[0].tolist()

# Realizar una predicción
url = f'{public_url}/predecir'
payload = {"caracteristicas": ejemplo}
response = requests.post(url, json=payload)

print("Resultado de la predicción:")
print(json.dumps(response.json(), indent=2))

## 5. Explorar la Documentación

Puedes acceder a la documentación interactiva de la API agregando `/docs` a la URL pública.

## 6. Limpieza

Cuando hayas terminado, asegúrate de cerrar el túnel ngrok:

In [ ]:
ngrok.kill()